In [ ]:
import time

import bs4
from bs4 import BeautifulSoup

from selenium import webdriver
import json

# Get a specific type of news

download chromedriver version similar to your chrome browser   
https://sites.google.com/a/chromium.org/chromedriver/downloads

installation:
```
$ unzp chromedriver_linux64.zip
$ sudo mv chromedriver /usr/bin/chromedriver
$ sudo chown root:root /usr/bin/chromedriver
$ sudo chmod +x /usr/bin/chromedriver
```

In [ ]:
base_site = "https://data.covid19.go.id/public/index.html"

options = webdriver.ChromeOptions()
options.add_argument("headless")

driver = webdriver.Chrome(executable_path="/usr/bin/chromedriver", 
                          options=options)
driver.get(base_site)

In [ ]:
soup_page = BeautifulSoup(driver.page_source, parser='html5lib')
soup_page_str = str(soup_page)
#print(type(soup_page))
#print(soup_page.prettify())
soup_page_str

In [ ]:
soup_page = BeautifulSoup(soup_page_str, parser='html5lib')

time_stamp = [data.get_text().split()[-1] 
              for data in soup_page.find_all('span', attrs={'class': 'pull-right'}) 
                  if 'Tanggal' in data.get_text()]
time_stamp

In [ ]:
print(f"Dataset is up to: {time_stamp[0]}")

In [ ]:
headlines = [[headline_data.find('h3').string] + [data.string for data in headline_data.find_all('b')]
             for headline_data in soup_page.find_all('div', attrs={'class': 'col-md-3'})]
headlines = {label.lower(): [total_case, increment.lower()]
             for total_case, label, increment in headlines}
headlines

In [ ]:
for headline in headlines:
    label, val = list(headline.items())[0]
    print(f"{label:>16s}: {val[0]} orang ({val[1]})")

In [ ]:
provinces_data = {data.find('b').get_text().strip().lower()\
                   : data.find('span').string[15:]
                  for data in soup_page.find_all('div', attrs={'class': 'wrapDetailInfo'})}
provinces_data

In [ ]:
provinces_data['jawa timur']

In [ ]:
not None

In [ ]:
data = {'time_stamp': time_stamp, 
        'headlines': headlines,
        'provinces_data': provinces_data}

json_data = json.dumps(data, indent=4)

with open("covid_19.json", "w") as f:
    f.write(json_data)

In [ ]:
with open("covid_19.json", "r") as f:
    data = f.read()
    
print(json.loads(data))

In [ ]:
from threading import Thread
from multiprocessing import Process, Manager

class WorkerFunc(object):
    def __init__(self, queue_dict):
        self.queue_dict = queue_dict
        
    def worker(self, bs):
        a = [bs, "a", 1]
        self.queue_dict[0] = a
        time.sleep(3)
        
        
class CountdownTask: 
    def __init__(self): 
        self._running = True
      
    def terminate(self): 
        self._running = False
      
    def counter(self): 
        while self._running: 
            print(".", end="") 
            time.sleep(0.5) 

if __name__ == '__main__':
    manager = Manager()

    return_dict = manager.dict()
    print(return_dict.values())

    counter_fun = CountdownTask() 
    counter_proc = Thread(target=counter_fun.counter) 
    counter_proc.start() 
    #print(os.getpid())
    
    # Signal termination 
    worker_fun = WorkerFunc(return_dict)
    p1 = Process(target=worker_fun.worker,args=(bs4.BeautifulSoup, ))
    p1.start()
    p1.join()
    if not p1.is_alive():
        counter_fun.terminate()
        counter_proc.join()

    print(return_dict.values())

In [2]:
import bs4
import sys
import time

from threading import Thread
from multiprocessing import Process, Manager

from bs4 import BeautifulSoup

from selenium import webdriver

#sys.setrecursionlimit(1000)

class WorkerTask(object):
    def __init__(self, queue_dict):
        self.queue_dict = queue_dict
        
    def worker(self):
        base_site = "https://data.covid19.go.id/public/index.html"
        
        options = webdriver.ChromeOptions()
        options.add_argument("headless")
        driver = webdriver.Chrome(executable_path="/usr/bin/chromedriver", options=options)
        driver.get(base_site)
        
        soup_page = BeautifulSoup(driver.page_source, parser="html5lib", features="lxml")
        
        self.queue_dict[0] = str(soup_page)
        #time.sleep(3)
        
        
class ProgressTask: 
    def __init__(self): 
        self._running = True
      
    def terminate(self): 
        self._running = False
      
    def print_point(self): 
        while self._running: 
            sys.stdout.write(".") 
            time.sleep(1) 


def scrap_covid19_go_id():
    queue_dict = Manager().dict()
    print(queue_dict.values())

    progress_ins = ProgressTask() 
    progress_proc = Thread(target=progress_ins.print_point) 
    progress_proc.start() 
    #print(os.getpid())
    
    # Signal termination 
    worker_ins = WorkerTask(queue_dict)
    worker_proc = Process(target=worker_ins.worker)
    worker_proc.start()
    worker_proc.join()
    
    if not worker_proc.is_alive():
        progress_ins.terminate()
        progress_proc.join()

    #print(queue_dict.values())
    soup_page = BeautifulSoup(queue_dict.values()[0], parser="lxml", features="lxml")
    
    return soup_page

            
if __name__ == '__main__':
    soup_page = scrap_covid19_go_id()
    #print(soup_page.prettify())
    #print(type(soup_page))

[]
.......

In [ ]:
!python web-scrapping.py -f True